# Beth Dataset Generator

The purpose of this notebook is to process the document files Beth sent accross into CSV files.

Before using this script, we did:

1. We validated all of Beth's file were in a `{title}\n{paragraph}`. Any files not following that format were adapted to follow the format. 
2. All `.doc` files were converted to a standard `.docx` file via: ``textutil -convert docx *.doc``
3. All `.mp3` files were converted to `.wav` files via: ``find . -name "*.mp3" -exec bash -c 'ffmpeg -i "{}" "${0/.mp3}.wav"' {} \;``
4. Remove files with spanish in them by searching the documents for common spanish words like "el,la": https://www.happyhourspanish.com/learning-efficiently-start-with-the-250-most-common-spanish-words/
5. Remove unverbalized copy that does not represent a section title. 

In [1]:
from pathlib import Path

destination = Path('./../../_data/02 Beth/') # Files to write processed dataset
destination.mkdir(exist_ok=True)

source = Path('./../../_data/01 Beth/') # Source destination
source.is_dir()

True

## Check Invariants 

Check that all the files in `source` are accounted for with the below patterns.

In [2]:
all_files = set([path for path in source.glob('*') if path.is_file()])
accounted_for_files = set(list(source.glob('*.docx')) + list(source.glob('*.doc')) + 
                          list(source.glob('*.wav')) + list(source.glob('*.mp3')))
all_files.difference(accounted_for_files)

{PosixPath('../../_data/01 Beth/.DS_Store'),
 PosixPath('../../_data/01 Beth/~$ng_FloodlightCamMotionDetectionSettings_082817.rtf'),
 PosixPath('../../_data/01 Beth/~$ng_HowToInslallRingVideoDoorbellPro_111417.rtf'),
 PosixPath('../../_data/01 Beth/~$ng_HowToInstallSpotlightCamMount_110917.rtf'),
 PosixPath('../../_data/01 Beth/~$ng_HowToJoinRingNeighborhoodWithoutADevice_110917.rtf'),
 PosixPath('../../_data/01 Beth/~$ng_HowToSolvePowerProblemsWithRingVideoDoorbellPro_111417.rtf')}

Check that all `.docx` files have an associated `.wav` file. (f.y.i. we fixed any naming issues causing this invariant to fail)

In [4]:
text_file_stems = set([path.stem for path in source.glob('*.docx')])
audio_file_stems = set([path.stem for path in source.glob('*.wav')])
print(text_file_stems.difference(audio_file_stems))
print(audio_file_stems.difference(text_file_stems))

set()
set()


## Generate Script

Finally, we preprocess all the text files into CSV files. 

In [5]:
from IPython.display import FileLink
from IPython.display import Markdown
import json
import docx
import pandas
import re
import pandas as pd
import shutil

def remove_parentheses(text):
    """ Remove paranethesis and / or brackets in text.
    
    Example: 
    
        >>> text = "Statement 7:  (repeating Statement 3)"
        >>> remove_parentheses(text)
        Statement 7:
        
    Args:
        text (str)

    Returns:
        str
    """
    return re.sub("[\(\[].*?[\)\]]", "", text).strip()

def is_title(text):
    """ Return `True` if text is a "title".
    
    Example titles:
      Statement 2:
      For Spanish:
      Statement 4A:
      Day Greeting:
      Close message:
      Statement:  10:
      :15 Spot\u2014Ready?
      Phone Greeting:
      On Hold Program:
      After pressing 2:
      Holiday Greeting:
      Prompt2a\u2013Press3Menu
      Main Greeting Menu:
      Statement 4:  (Blue)
      Option 1 \u2013 No Answer:
      Testimonial 1: (BLUE)
      Callback Confirmation:
      Secondary IVR Greeting:
      Prompt12-FunshoVoicemail
      New Statement 2: (for post holiday program)_
      Prompt9-Press2MenuOpenNationalHolidayM-F9AM-8PM
    
    Args:
        text (str)

    Returns:
        bool
    """
    text = remove_parentheses(text)
    if text[-1] == ':' and len(text) < 50: 
        return True 
    # Match:
    # 6.
    # 4.
    # 5.
    if len(text) < 5:
        return True
    # Match
    # Ready?
    # Statement3
    # Statement 5
    # :15 Spot\u2014Ready?
    # Voicemail Greeting.
    # Prompt1\u2013ThankYouForCalling
    if len(text) < 25 and any([s in text for s in ['Ready', 'Statement', 'Greeting', 'Prompt']]):
        return True
    # Matches: 
    # Prompt10-Press2MenuOpenNationalHolidayM-F7AM-8PM
    # Prompt 5b- AssistingAnotherPatient-Hold Message
    # Prompt2b\u2013Press3Menu- Monarch Physiotherapy
    if re.match(r"prompt[\s]{0,1}[0-9]+.*", text, re.IGNORECASE):
        return True
    return False

def is_noise(text):
    """ Return `True` if text is "noise".
    
    Example noise:
      ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
      telephoneonhold.com\t1-888-321-8477
      PAGE 
      .
      
    Args:
        text (str)

    Returns:
        bool
    """
    if len(text) == 0:
        return True
    if not re.search(r"[a-zA-Z0-9]", text): # Has no letters in the text
        return True
    if 'PAGE' in text: # Typically seen as the last text in a script
        return True
    if 'telephoneonhold.com\t1-888-321-8477' in text: # One off case
        return True
    if '(Dr. Jack (grin year))' in text: # One off case
        return True
    return False

def has_white_text(paragraph):
    """ Return `True` if paragraph contains white text. 
    
    Args:
        paragraph (docx.text.paragraph.Paragraph)

    Returns:
        bool
    """
    if 'FFFFFF' in set([str(r.font.color.rgb) for r in paragraph.runs]):
        display(Markdown('Whited Out Text: "%s"' % paragraph.text))
        return True
    return False
        
    
def strip_quotes(text):
    """ Strip quotes if ``text`` has quotes on both sides.
    
    Args:
        text (str)
        
    Returns:
        str
    """
    if (text[0] == '“' or text[0] == '"') and (text[-1] == '"' or text[-1] == '”'):
        return text[1:-1]
    return text
    
# Flatten a 2-d list into a 1-d list
# Inspired by: https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
flatten = lambda l: [item for sublist in l for item in sublist]
    
text_files = [path for path in source.glob('*.docx')]
audio_files = [path for path in source.glob('*.wav')]

# For inspecting results accross the entire dataset
all_content = []
all_title = []
all_noise = []

for text_file, audio_file in zip(sorted(text_files), sorted(audio_files)):
    display(FileLink(text_file))
    
    document = docx.Document(text_file)
    # Split by newline, filter white text, filter empty text
    paragraphs = flatten([p.text.split('\n') for p in document.paragraphs 
                          if len(p.text.strip()) > 0 and not has_white_text(p)])
    # Split by 7 spaces or more and strip (This number was set by trial and error)
    # NOTE: In the Beth dataset, instead of a newline there is a bunch of spaces sometimes.
    paragraphs = [re.split(r'\s{7,}', p.strip()) for p in paragraphs]
    for p in paragraphs:
        if len(p) > 1:
            display(Markdown('Split by Spaces: %s' % p))
    paragraphs = flatten(paragraphs)
    
    # Create table with title and content columns.
    rows = []
    for paragraph in paragraphs:
        paragraph = paragraph.strip()
        if is_noise(paragraph):
            display(Markdown('Skipped Paragraph: "%s"' % paragraph))
            all_noise.append(paragraph)
        elif is_title(paragraph):
            if len(paragraph) > 75: # Determined by trial and error
                display(Markdown('Long Title: "%s"' % paragraph))
                
            rows.append({'Title': paragraph, 'Content': []})
            all_title.append(paragraph)
        else:
            if len(paragraph) < 75: # Determined by trial and error
                display(Markdown('Short Content: "%s"' % paragraph))
                
            paragraph = strip_quotes(paragraph) 
            all_content.append(paragraph)
            if len(rows) == 0:
                rows = [{'Title': '', 'Content': [paragraph]}]
            else:   
                rows[-1]['Content'].append(paragraph)
    
    for row in rows:
        row['Content'] = '\n'.join(row['Content'])
    
    rows = [r for r in rows if len(r['Content']) > 0]
    
    df = pd.DataFrame(rows)
    df.to_csv(str(destination / (text_file.stem + '.csv')), index=False)
    shutil.copy(audio_file, str(destination / audio_file.name))
    display(Markdown('-' * 50))

/Users/michaelp/Code/WellSaidLabs/_data/01 Beth/A1BondedTermite_111315.docx

Skipped Paragraph: "PAGE"

--------------------------------------------------

/Users/michaelp/Code/WellSaidLabs/_data/01 Beth/AAHFluidPower_031918.docx

--------------------------------------------------

...

Skipped Paragraph: "telephoneonhold.com	1-888-321-8477"

--------------------------------------------------

/Users/michaelp/Code/WellSaidLabs/_data/01 Beth/WhitesCanyonFord_Lincoln_112316.docx

Skipped Paragraph: "PAGE"

--------------------------------------------------

## QA Script

In [6]:
print(set(all_noise))

{'', 'PAGE  1', ':', '++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++', '++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++', '(Dr. Jack (grin year))', '+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++', '++++++++++++++++++++++++++++++++++++++', 'telephoneonhold.com\t1-888-321-8477', 'PAGE', '.'}


In [7]:
print(json.dumps(sorted(list(set(all_title)), key=lambda h: len(h)), indent=2))

[
  "5.",
  "6.",
  "4.",
  "Ready?",
  "Hours:",
  "Prompt:",
  "Address:",
  "Prompt 3:",
  "Prompt 2:",
  "Prompt 1:",
  "Location:",
  "Statement3",
  "Statement 6",
  "Statement 5",
  "Statement 7",
  "Statement 6:",
  "For Spanish:",
  "Statement 5:",
  "Statement 4:",
  "Statement 2:",
  "Statement 7:",
  "Statement 3:",
  "Statement 8:",
  "Statement 9:",
  "Statement 1:",
  "Statement 3a:",
  "Day Greeting:",
  "Statement 4a:",
  "Statement 10:",
  "Statement 2a:",
  "Statement 2b:",
  "Statement 12:",
  "Statement 4A:",
  "Statement 3A:",
  "Statement 13:",
  "Statement 14:",
  "Statement 1B:",
  "Statement 5a:",
  "Statement 6A:",
  "Statement 5A:",
  "Statement 2A:",
  "Statement 11:",
  "Statement 1a:",
  "Close message:",
  "Phone Greeting:",
  "Statement:  10:",
  ":15 Spot\u2014Ready?",
  "Night Greeting:",
  "Closed Greeting:",
  "On Hold Program:",
  "Welcome Greeting:",
  "After pressing 3:",
  "Daytime Greeting:",
  "Greeting Message:",
  "Holiday Greeting:",
  "Aft

In [8]:
print(json.dumps(sorted(list(set(all_content)), key=lambda h: len(h)), indent=2))

[
  "-January.",
  "necessary.\u201d",
  "<little pause>",
  "will assist you.",
  "For Bayo, Press 1",
  "For Funsho, Press 1",
  "You\u2019re almost done!",
  "To reach Bayo Press 1",
  "For Brad Hall press 2.",
  "To reach Funsho Press 1",
  "For Casey Chase press 3.",
  "For Aaron Tripp press 1.",
  "Then, tap Motion Snooze.",
  "For insurance press two.",
  "For Tim Lambert press 7.",
  "To reach Payroll, press 6",
  "For Chiropractor, Press 4",
  "To reach Finance, press 7",
  "For Mike Hopkins press 4.",
  "For Scott Stuart press 6.",
  "For Jay in parts press 4.",
  "For Scott Roberts press 5.",
  "For Bill in parts press 3.",
  "For forms, please press 4.",
  "...set your desired angle\u2026",
  "For Kelly in parts press 1.",
  "For Physiotherapist, Press 2",
  "to reach Recruiting, press 5",
  "to reach Retail Ops, press 4",
  "...then re-tighten the screw.",
  "For the Chiropractor, Press 4",
  "With Ring you\u2019re always home.",
  "For Weston in IT/Web press 4.",
  "...an